## Data Simulation for future covariates to predict energy prices 


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from darts import TimeSeries
from darts.models import ExponentialSmoothing


In [2]:
# Import the data 
def load_and_prepare_data(file_path):
    """
    Load energy prices data from a CSV file, ensure chronological order, and convert 'Date' to datetime.
    """
    df = pd.read_csv(file_path)
    df.sort_values('Date', inplace=True)
    df.set_index('Date', inplace=True)
    df = pd.DataFrame(df)
    return df

In [3]:
# Import the data
df = load_and_prepare_data('../../data/Final_data/final_data_july.csv')

# Reset the index
df.reset_index(inplace=True)
# Ensure the 'Date' column is properly parsed and set as the index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Ensure the data has a daily frequency (Filling missing dates if necessary)
df = df.asfreq('D')  # This makes sure the data has a daily frequency

In [4]:
df

,Day_ahead_price (€/MWh),Solar_radiation (W/m2),Wind_speed (m/s),Temperature (°C),Biomass (GWh),Hard_coal (GWh),Hydro (GWh),Lignite (GWh),Natural_gas (GWh),Other (GWh),Pumped_storage_generation (GWh),Solar_energy (GWh),Wind_offshore (GWh),Wind_onshore (GWh),Net_total_export_import (GWh),BEV_vehicles,Oil_price (EUR),TTF_gas_price (€/MWh),Nuclear_energy (GWh)
Date,,,,,,,,,,,,,,,,,,,
2012-01-01,18.19,14.75,4.95,8.39,98.605,108.454,51.011,325.337,188.811,54.040,19.314,6.263,3.404,235.467,54.662,6,99.64,21.10,250.979
2012-01-02,33.82,15.12,5.00,7.41,98.605,222.656,51.862,343.168,229.293,54.166,28.892,6.312,3.350,231.772,-64.477,6,100.04,20.00,258.671
2012-01-03,35.03,31.88,7.77,5.23,98.605,162.204,48.851,336.773,241.297,53.518,21.072,24.226,7.292,504.484,-35.078,6,100.44,20.90,271.495
2012-01-04,32.16,25.21,8.04,4.78,98.605,189.633,47.101,323.976,252.289,52.194,28.300,14.157,7.828,541.528,22.924,6,103.15,21.40,270.613
2012-01-05,20.35,13.46,9.98,4.23,98.605,175.733,45.854,327.502,259.018,52.179,31.887,4.728,8.280,572.819,35.618,6,103.92,21.30,287.555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-24,66.61,225.04,3.47,17.54,110.007,43.469,85.857,199.246,194.291,54.026,20.934,325.285,49.360,179.921,-168.705,992,75.75,32.63,0.000
2024-07-25,78.34,272.71,2.12,17.85,110.410,50.676,82.632,195.983,209.610,52.963,18.766,394.116,51.053,42.885,-194.496,992,76.36,31.70,0.000
2024-07-26,93.04,172.33,2.60,19.09,110.852,42.333,79.531,205.273,205.773,52.616,19.081,256.246,40.449,129.267,-241.786,993,75.21,32.20,0.000


In [5]:
# Define the future covariates columns from your dataframe
covariates = ['Solar_radiation (W/m2)', 'Wind_speed (m/s)', 'Temperature (°C)', 
                             'Biomass (GWh)', 'Hard_coal (GWh)', 'Hydro (GWh)', 'Lignite (GWh)', 
                             'Natural_gas (GWh)', 'Other (GWh)', 'Pumped_storage_generation (GWh)', 
                             'Solar_energy (GWh)', 'Wind_offshore (GWh)', 'Wind_onshore (GWh)', 
                             'Net_total_export_import (GWh)']

future_covariates_columns = df[covariates]

In [7]:
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly
import plotly.graph_objs as go

# Load your data (adjust the path and data columns)
# Prepare data for Prophet
df_prophet = pd.DataFrame({
    'ds': df.index, 
    'y': df["Solar_radiation (W/m2)"]
})

# Initialize the Prophet model
m = Prophet(
    seasonality_mode='additive',  # Additive seasonality is not always necessary
    yearly_seasonality=20,
    weekly_seasonality=False,  # Disable weekly seasonality if not needed
    daily_seasonality=False    # Disable daily seasonality
)

# Fit the model
m.fit(df_prophet)

# Make future predictions for the next 2 years (730 days)
future = m.make_future_dataframe(periods=730)
forecast = m.predict(future)

# Use Plotly for visualization 
fig = plot_plotly(m, forecast)

# Add additional layout elements if necessary
fig.update_layout(
    title="Forecasted Solar radiation",
    xaxis_title="Date",
    yaxis_title="Solar_radiation (W/m2)",
    legend=dict(
        x=1, y=1,
        xanchor="right",
        yanchor="top"
    ),
    template="plotly_white"
)

# Show the Plotly plot
fig.show()


09:20:22 - cmdstanpy - INFO - Chain [1] start processing
09:20:22 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2012-01-01,124.420345,-33.012187,66.858169,124.420345,124.420345,-104.808872,-104.808872,-104.808872,-104.808872,-104.808872,-104.808872,0.0,0.0,0.0,19.611473
1,2012-01-02,124.423112,-26.246962,63.691026,124.423112,124.423112,-105.065164,-105.065164,-105.065164,-105.065164,-105.065164,-105.065164,0.0,0.0,0.0,19.357948
2,2012-01-03,124.425880,-29.217964,71.124331,124.425880,124.425880,-105.500762,-105.500762,-105.500762,-105.500762,-105.500762,-105.500762,0.0,0.0,0.0,18.925118
3,2012-01-04,124.428648,-29.322555,68.766030,124.428648,124.428648,-106.047125,-106.047125,-106.047125,-106.047125,-106.047125,-106.047125,0.0,0.0,0.0,18.381523
4,2012-01-05,124.431415,-29.541215,66.591888,124.431415,124.431415,-106.617251,-106.617251,-106.617251,-106.617251,-106.617251,-106.617251,0.0,0.0,0.0,17.814164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5318,2026-07-24,122.421174,180.760992,273.956116,121.448411,123.453936,103.028498,103.028498,103.028498,103.028498,103.028498,103.028498,0.0,0.0,0.0,225.449672
5319,2026-07-25,122.413457,170.622004,273.775243,121.438650,123.449084,100.025163,100.025163,100.025163,100.025163,100.025163,100.025163,0.0,0.0,0.0,222.438619
5320,2026-07-26,122.405739,169.286335,269.783458,121.428888,123.444232,96.825260,96.825260,96.825260,96.825260,96.825260,96.825260,0.0,0.0,0.0,219.230999
5321,2026-07-27,122.398022,167.249066,266.915662,121.419127,123.439343,93.604648,93.604648,93.604648,93.604648,93.604648,93.604648,0.0,0.0,0.0,216.002670


# Hyperparametertuning

In [26]:
# Install needed packages
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly, plot_cross_validation_metric
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import itertools
from prophet.diagnostics import cross_validation, performance_metrics
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import time

In [27]:
# Import the data
df = load_and_prepare_data('../../data/Final_data/final_data.csv')

# Reset the index
df = df.reset_index()

#rename date and Day_ahead_price columns to ds and y
df = df.rename(columns={'Date': 'ds', "Solar_radiation (W/m2)": 'y'})


In [ ]:
import itertools
from tqdm import tqdm
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import time

# Define the hyperparameter grid
param_grid = {  
    'changepoint_prior_scale': [0.05, 0.1],
    'seasonality_prior_scale': [5],
    "holidays_prior_scale": [5],
    'seasonality_mode': ['additive', 'multiplicative'],
}

param_dist = {  
    'changepoint_prior_scale': [0.01, 0.05, 0.1, 0.25, 0.5],
    'seasonality_prior_scale': [1, 5, 10],
    'holidays_prior_scale': [1, 5, 10],
    'seasonality_mode': ['additive', 'multiplicative'],
}


# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_dist.values())]

# Function to run Prophet with a specific parameter set
def evaluate_params(params):
    try:
        # Fit model with given params
        m = Prophet(**params).fit(df)
        df_cv = cross_validation(m, initial='730 days', period='180 days', horizon='365 days')
        df_p = performance_metrics(df_cv, rolling_window=0.1)

        # Calculate the average RMSE across all horizons
        avg_rmse = df_p['rmse'].mean()

        return params, avg_rmse
    except Exception as e:
        # Handle any exceptions that occur during evaluation
        print(f"Error occurred with parameters {params}: {e}")
        return params, float('inf'), pd.Timedelta(seconds=0)

# Start the timer
start_time = time.time()

# Use ThreadPoolExecutor to parallelize the evaluation
with ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(evaluate_params, all_params), total=len(all_params)))

# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time
print(f"Total execution time: {elapsed_time:.2f} seconds")

# Extract the RMSEs, horizons, and corresponding params
rmses = [result[1] for result in results]
tuning_results = pd.DataFrame([result[0] for result in results])
tuning_results['average_rmse'] = rmses

# Output the tuning results
print(tuning_results)


In [ ]:
# Python
best_params = all_params[np.argmin(rmses)]
print(best_params)
